In [64]:
import sys, xgboost as xgb, sklearn
print("Python:", sys.executable)
print("xgboost:", xgb.__version__)
print("scikit-learn:", sklearn.__version__)

Python: /usr/local/bin/python3
xgboost: 3.0.5
scikit-learn: 1.7.2


In [65]:
# Run these in a notebook cell (so it targets the active kernel)
%pip install -U "xgboost>=2.0.0" "scikit-learn>=1.3" "numpy>=1.23" "scipy>=1.7"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 20.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
  Attempting uninstall: scipy━━━━━━━━━━━━━━━━━━━ 0/2 [numpy]
    Found existing installation: scipy 1.15.0 0/2 [numpy]
    Uninstalling scipy-1.15.0:━━━━━━━━━━━━━━ 0/2 [numpy]
      Successfully uninstalled scipy-1.15.0━ 0/2 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scipy]32m1/2 [scipy]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import xgboost as xgb, sklearn, inspect
print("xgboost:", xgb.__version__)
print("scikit-learn:", sklearn.__version__)
print("early_stopping_rounds in XGBRegressor.fit:",
      'early_stopping_rounds' in inspect.signature(xgb.XGBRegressor.fit).parameters)
print("eval_metric in XGBRegressor.__init__:",
      'eval_metric' in inspect.signature(xgb.XGBRegressor.__init__).parameters)

xgboost: 3.0.5
scikit-learn: 1.7.2
early_stopping_rounds in XGBRegressor.fit: False
eval_metric in XGBRegressor.__init__: False


In [38]:
# %pip install optuna
%pip install -U scikit-learn


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import warnings, json
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from catboost import CatBoostRegressor, Pool

import optuna
import xgboost as xgb
from xgboost.callback import EarlyStopping as XgbEarlyStopping




In [8]:
# Config

RANDOM_STATE = 42
N_SPLITS = 5
EARLY_STOP = 200
VERBOSE_TUNE = 0

CV_KIND = 'kfold'
SHUFFLE = True 

np.random.seed(RANDOM_STATE)

In [9]:
TRAIN_PATH = "./data/train_combined.csv"
TEST_PATH = "./data/test_combined.csv"
ID_COL = "id"

TARGETS = ["Y1", "Y2"]
DROP_COLS = [ID_COL]


In [10]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

feature_cols = [c for c in df_train.columns if c not in set(TARGETS + DROP_COLS)]
cat_cols = []
num_cols = [c for c in feature_cols]

X = df_train[feature_cols].reset_index(drop=True)
test_X = df_test[feature_cols].reset_index(drop=True)
Y = {t: df_train[t].reset_index(drop=True) for t in TARGETS}
cat_idx = []

In [11]:
def get_cv():
    if CV_KIND == "timeseries":
        return TimeSeriesSplit(n_splits=N_SPLITS)
    return KFold(n_splits=N_SPLITS, shuffle=SHUFFLE, random_state=RANDOM_STATE)

cv = get_cv()

In [22]:
def tune_catboost_for_target(y_col, n_trials=60, use_gpu=False):
    y = Y[y_col]

    def objective(trial):
        params = {
            "loss_function": "RMSE",
            "eval_metric": "R2",
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 20.0, log=True),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 5.0),
            "random_strength": trial.suggest_float("random_strength", 0.0, 2.0),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 64),
            "n_estimators": trial.suggest_int("n_estimators", 800, 2500, step=100),
            "allow_writing_files": False,
            "random_state": RANDOM_STATE,
            "verbose": VERBOSE_TUNE
        }
        if use_gpu:
            params["task_type"] = "GPU"

        scores = []
        for fold, (tr_idx, va_idx) in enumerate(cv.split(X, y)):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            model = CatBoostRegressor(**params)
            model.fit(
                X_tr, y_tr,
                cat_features=[],
                eval_set=(X_va, y_va),
                early_stopping_rounds=EARLY_STOP,
            )
            pred = model.predict(X_va)
            scores.append(r2_score(y_va, pred))

            trial.report(float(np.mean(scores)), step=fold)
            if trial.should_prune():
                raise optuna.TrialPruned()
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE), pruner=optuna.pruners.MedianPruner(n_warmup_steps=1))

    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    return study


In [23]:
N_TRIALS_CAT = 60
studies = {}
for tgt in TARGETS:
    print(f"=== Tuning CatBoost for {tgt} ===")
    st = tune_catboost_for_target(tgt, n_trials=N_TRIALS_CAT, use_gpu=False)
    print(f"[{tgt}] Best CV R2: {st.best_value:.5f}")
    print(f"[{tgt}] Best Params: {st.best_trial.params}\n")
    studies[tgt] = st

[I 2025-09-22 12:10:01,123] A new study created in memory with name: no-name-8b97ccbb-231f-4efb-8154-8e326f3c85a3


=== Tuning CatBoost for Y1 ===


  0%|          | 0/60 [00:00<?, ?it/s]

0:	learn: 0.2589248	test: 0.2701131	best: 0.2701131 (0)	total: 62.5ms	remaining: 56.2s
200:	learn: 0.8345391	test: 0.7663280	best: 0.7665486 (183)	total: 541ms	remaining: 1.88s
400:	learn: 0.8605668	test: 0.7656530	best: 0.7669320 (208)	total: 1.06s	remaining: 1.32s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7669320283
bestIteration = 208

Shrink model to first 209 iterations.
0:	learn: 0.2582605	test: 0.2761816	best: 0.2761816 (0)	total: 3.35ms	remaining: 3.02s
200:	learn: 0.8391467	test: 0.7509585	best: 0.7570974 (42)	total: 519ms	remaining: 1.8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7570974249
bestIteration = 42

Shrink model to first 43 iterations.
0:	learn: 0.2593947	test: 0.2474236	best: 0.2474236 (0)	total: 3.12ms	remaining: 2.8s
200:	learn: 0.8316439	test: 0.7416594	best: 0.7441175 (102)	total: 481ms	remaining: 1.67s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7441175008
bestIteration = 102

Shrin

Best trial: 0. Best value: 0.753633:   2%|▏         | 1/60 [00:04<04:35,  4.67s/it]

...

1099:	learn: 0.8963642	test: 0.7817703	best: 0.7817843 (1096)	total: 4.12s	remaining: 0us

bestTest = 0.7817843011
bestIteration = 1096

Shrink model to first 1097 iterations.
[I 2025-09-22 13:02:47,285] Trial 57 finished with value: 0.7923055893273497 and parameters: {'depth': 8, 'learning_rate': 0.046954473785406625, 'l2_leaf_reg': 2.002219698535936, 'bagging_temperature': 3.8310955759366006, 'random_strength': 1.301863740392971, 'min_data_in_leaf': 22, 'n_estimators': 1100}. Best is trial 44 with value: 0.7941634916988844.
0:	learn: 0.0682054	test: 0.0655368	best: 0.0655368 (0)	total: 2.98ms	remaining: 7.46s
200:	learn: 0.8092192	test: 0.7880761	best: 0.7880761 (200)	total: 471ms	remaining: 5.39s
400:	learn: 0.8394956	test: 0.8000054	best: 0.8000054 (400)	total: 957ms	remaining: 5.01s
600:	learn: 0.8552710	test: 0.8041036	best: 0.8041036 (600)	total: 1.43s	remaining: 4.53s
800:	learn: 0.8668733	test: 0.8061679	best: 0.8061679 (800)	total: 1.91s	remaining: 4.05s
1000:	learn: 0.875777

Best trial: 44. Best value: 0.794163:  98%|█████████▊| 59/60 [33:06<00:23, 23.12s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7867251431
bestIteration = 2106

Shrink model to first 2107 iterations.
[I 2025-09-22 13:03:05,336] Trial 58 finished with value: 0.793382730038051 and parameters: {'depth': 6, 'learning_rate': 0.06347198696947666, 'l2_leaf_reg': 1.7103672585486007, 'bagging_temperature': 3.4688234271021043, 'random_strength': 1.486813412398972, 'min_data_in_leaf': 26, 'n_estimators': 2500}. Best is trial 44 with value: 0.7941634916988844.
0:	learn: 0.0724034	test: 0.0695816	best: 0.0695816 (0)	total: 2.85ms	remaining: 7.13s
200:	learn: 0.8126062	test: 0.7839790	best: 0.7839790 (200)	total: 488ms	remaining: 5.58s
400:	learn: 0.8412843	test: 0.7967117	best: 0.7967117 (400)	total: 982ms	remaining: 5.14s
600:	learn: 0.8577509	test: 0.8006276	best: 0.8007141 (594)	total: 1.46s	remaining: 4.6s
800:	learn: 0.8687424	test: 0.8021166	best: 0.8021166 (800)	total: 1.94s	remaining: 4.11s
1000:	learn: 0.8780841	test: 0.8038183	best: 0.8038347 (99

Best trial: 44. Best value: 0.794163: 100%|██████████| 60/60 [33:24<00:00, 33.41s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7849116198
bestIteration = 1367

Shrink model to first 1368 iterations.
[I 2025-09-22 13:03:23,358] Trial 59 finished with value: 0.7912232689017242 and parameters: {'depth': 6, 'learning_rate': 0.06752602184936711, 'l2_leaf_reg': 1.756940662706984, 'bagging_temperature': 3.4558254284747303, 'random_strength': 1.4691890908260257, 'min_data_in_leaf': 18, 'n_estimators': 2500}. Best is trial 44 with value: 0.7941634916988844.
[Y2] Best CV R2: 0.79416
[Y2] Best Params: {'depth': 8, 'learning_rate': 0.04557837932439958, 'l2_leaf_reg': 1.003216622307434, 'bagging_temperature': 2.8629839499568, 'random_strength': 0.30214757243346213, 'min_data_in_leaf': 34, 'n_estimators': 1300}



In [26]:
# print(studies["Y1"].best_trial.params)
# print(studies["Y2"].best_trial.params)

cat_best_params = {"Y1": {'depth': 8, 'learning_rate': 0.040727522856511406, 'l2_leaf_reg': 1.0333160687802394, 'bagging_temperature': 3.8381299885039564, 'random_strength': 1.177785615607334, 'min_data_in_leaf': 1, 'n_estimators': 1400},
                   "Y2": {'depth': 8, 'learning_rate': 0.04557837932439958, 'l2_leaf_reg': 1.003216622307434, 'bagging_temperature': 2.8629839499568, 'random_strength': 0.30214757243346213, 'min_data_in_leaf': 34, 'n_estimators': 1300}
                   }
"""
{'depth': 8, 'learning_rate': 0.040727522856511406, 'l2_leaf_reg': 1.0333160687802394, 'bagging_temperature': 3.8381299885039564, 'random_strength': 1.177785615607334, 'min_data_in_leaf': 1, 'n_estimators': 1400}
{'depth': 8, 'learning_rate': 0.04557837932439958, 'l2_leaf_reg': 1.003216622307434, 'bagging_temperature': 2.8629839499568, 'random_strength': 0.30214757243346213, 'min_data_in_leaf': 34, 'n_estimators': 1300}
"""

"\n{'depth': 8, 'learning_rate': 0.040727522856511406, 'l2_leaf_reg': 1.0333160687802394, 'bagging_temperature': 3.8381299885039564, 'random_strength': 1.177785615607334, 'min_data_in_leaf': 1, 'n_estimators': 1400}\n{'depth': 8, 'learning_rate': 0.04557837932439958, 'l2_leaf_reg': 1.003216622307434, 'bagging_temperature': 2.8629839499568, 'random_strength': 0.30214757243346213, 'min_data_in_leaf': 34, 'n_estimators': 1300}\n"

In [28]:
def train_oof_catboost(y_col, params, use_gpu=False):
    y = Y[y_col].values
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(test_X))
    models = []

    p = dict(params)
    p.update({
        "loss_function": "RMSE",
        "eval_metric": "R2",
        "allow_writing_files": False,
        "verbose": 0
    })
    if use_gpu:
        p["task_type"] = "GPU"
    
    for fold, (tr_idx, va_idx) in enumerate(get_cv().split(X, y)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        model = CatBoostRegressor(**p, random_state=RANDOM_STATE + fold)
        model.fit(
            X_tr, y_tr,
            cat_features = [],
            eval_set = (X_va, y_va),
            early_stopping_rounds=EARLY_STOP,
        )
        oof[va_idx] = model.predict(X_va)
        test_pred += model.predict(test_X) / N_SPLITS
        models.append(model)
        print(f"[{y_col}] Fold {fold+1}/{N_SPLITS} complete.")

    cv_r2 = r2_score(Y[y_col], oof)
    mse  = mean_squared_error(Y[y_col], oof)
    rmse = float(np.sqrt(mse))
    print(f"[OOF] {y_col}: R2={cv_r2:.5f} | RMSE={rmse:.5f}")
    return models, oof, test_pred, cv_r2

In [29]:
cat_models = {}
oof_preds = {}
test_preds = {}
cv_scores = {}

for tgt in TARGETS:
    best_params = cat_best_params[tgt]
    models, oof, test_p, r2cv = train_oof_catboost(tgt, best_params, use_gpu=False)
    cat_models[tgt] = models
    oof_preds[tgt] = oof
    test_preds[tgt] = test_p
    cv_scores[tgt] = r2cv

print("CV Scores:", cv_scores)

[Y1] Fold 1/5 complete.
[Y1] Fold 2/5 complete.
[Y1] Fold 3/5 complete.
[Y1] Fold 4/5 complete.
[Y1] Fold 5/5 complete.
[OOF] Y1: R2=0.75949 | RMSE=0.47603
[Y2] Fold 1/5 complete.
[Y2] Fold 2/5 complete.
[Y2] Fold 3/5 complete.
[Y2] Fold 4/5 complete.
[Y2] Fold 5/5 complete.
[OOF] Y2: R2=0.79279 | RMSE=0.42047
CV Scores: {'Y1': 0.7594855670843246, 'Y2': 0.7927941229916622}


In [43]:
submission = pd.DataFrame()
submission[ID_COL] = df_test[ID_COL]
for tgt in TARGETS:
    submission[tgt] = test_preds[tgt]

OUT_PATH = "submission_catboost_optuna.csv"
submission.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
submission.head()

Saved: submission_catboost_optuna.csv


,id,Y1,Y2
0,1,0.537011,-0.134606
1,2,-0.204837,-0.322359
2,3,-0.295162,-0.119522
3,4,-0.444189,0.123059
4,5,-0.936314,0.126030


now test xgb boost and blend both together  

In [14]:
def _get_best_iteration(bst):
    # Works across xgboost 1.x/2.x/3.x
    bi = getattr(bst, "best_iteration", None)
    if bi is None:
        # some builds store it in attributes dict
        try:
            bi = int(bst.attributes().get("best_iteration"))
        except Exception:
            bi = None
    return bi

def _predict_best(bst, dmatrix):
    bi = _get_best_iteration(bst)
    if bi is not None:
        # in 3.x use iteration_range instead of ntree_limit
        return bst.predict(dmatrix, iteration_range=(0, bi + 1))
    # fallback: use all trees
    return bst.predict(dmatrix)
    
def tune_xgb_for_target(y_col, n_trials=60, use_gpu=False):
    y = Y[y_col]

    def objective(trial):
        num_boost_round = trial.suggest_int("num_boost_round", 800, 2500, step=100)
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 20.0, log=True),
            "alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
            "lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "tree_method": "gpu_hist" if use_gpu else "hist",
            "seed": RANDOM_STATE,
        }
        
        scores = []
        for fold, (tr_idx, va_idx) in enumerate(get_cv().split(X, y)):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

            dtr = xgb.DMatrix(X_tr, label=y_tr)
            dva = xgb.DMatrix(X_va, label=y_va)

            bst = xgb.train(
                params=params,
                dtrain=dtr,
                num_boost_round=num_boost_round,
                evals=[(dva, "valid")],
                early_stopping_rounds=EARLY_STOP,
                verbose_eval=False,
            )

            pred = _predict_best(bst, dva)
            scores.append(r2_score(y_va, pred))

            trial.report(float(np.mean(scores)), step=fold)
            if trial.should_prune():
                raise optuna.TrialPruned()
            
        return float(np.mean(scores))
    
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE), pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),)
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)
    return study

In [15]:
N_TRIALS_XGB = 60

xgb_studies = {}
for tgt in TARGETS:
    print(f"=== Tuning XGBoost for {tgt} ===")
    st = tune_xgb_for_target(tgt, n_trials=N_TRIALS_XGB, use_gpu=False)
    print(f"[{tgt}] Best CV R2 (XGB): {st.best_value:.5f}")
    print(f"[{tgt}] Best Params (XGB): {st.best_trial.params}\n")
    xgb_studies[tgt] = st

[I 2025-09-22 13:56:53,635] A new study created in memory with name: no-name-8cd3cca8-b896-4862-9dec-c14d95e27c83


=== Tuning XGBoost for Y1 ===


Best trial: 0. Best value: 0.748939:   2%|▏         | 1/60 [00:01<01:35,  1.62s/it]

[I 2025-09-22 13:56:55,261] Trial 0 finished with value: 0.7489387194456079 and parameters: {'num_boost_round': 1400, 'learning_rate': 0.2536999076681771, 'max_depth': 8, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'min_child_weight': 1.5957084694148351, 'reg_alpha': 2.9152036385288193e-08, 'reg_lambda': 0.6245760287469887, 'gamma': 3.005575058716044}. Best is trial 0 with value: 0.7489387194456079.


Best trial: 1. Best value: 0.75844:   3%|▎         | 2/60 [00:17<09:35,  9.92s/it] 

...

[I 2025-09-22 14:20:27,725] Trial 57 finished with value: 0.7886562253637917 and parameters: {'num_boost_round': 2400, 'learning_rate': 0.02728064798737986, 'max_depth': 10, 'subsample': 0.7074120954146743, 'colsample_bytree': 0.9339438746582654, 'min_child_weight': 2.6853246969803193, 'reg_alpha': 0.00014630341116613404, 'reg_lambda': 8.79199029539623e-07, 'gamma': 0.5803983153964868}. Best is trial 54 with value: 0.7951590940416461.


Best trial: 54. Best value: 0.795159:  98%|█████████▊| 59/60 [14:12<00:13, 13.07s/it]

[I 2025-09-22 14:20:29,502] Trial 58 pruned. 


Best trial: 54. Best value: 0.795159: 100%|██████████| 60/60 [14:43<00:00, 14.73s/it]

[I 2025-09-22 14:21:01,004] Trial 59 finished with value: 0.7938019020574376 and parameters: {'num_boost_round': 1500, 'learning_rate': 0.010067979159965075, 'max_depth': 9, 'subsample': 0.7522782566662983, 'colsample_bytree': 0.8607937656483744, 'min_child_weight': 5.488694988397482, 'reg_alpha': 1.0478950898932234e-05, 'reg_lambda': 0.020717890024278828, 'gamma': 0.24944600225524166}. Best is trial 54 with value: 0.7951590940416461.
[Y2] Best CV R2 (XGB): 0.79516
[Y2] Best Params (XGB): {'num_boost_round': 2300, 'learning_rate': 0.01354184424412271, 'max_depth': 9, 'subsample': 0.7339248014853161, 'colsample_bytree': 0.7407604054011988, 'min_child_weight': 1.392834443288169, 'reg_alpha': 1.693305529699566e-05, 'reg_lambda': 3.897250984223176e-06, 'gamma': 0.16970161657112237}



In [ ]:
print(xgb_studies["Y1"].best_trial.params)
print(xgb_studies["Y2"].best_trial.params)

"""
{'num_boost_round': 900, 'learning_rate': 0.011387425172293873, 'max_depth': 9, 'subsample': 0.5642895080936234, 'colsample_bytree': 0.5581569764539639, 'min_child_weight': 19.703853846581026, 'reg_alpha': 3.336634822084743e-07, 'reg_lambda': 1.129627556934382e-07, 'gamma': 0.4896075810838035}
{'num_boost_round': 2300, 'learning_rate': 0.01354184424412271, 'max_depth': 9, 'subsample': 0.7339248014853161, 'colsample_bytree': 0.7407604054011988, 'min_child_weight': 1.392834443288169, 'reg_alpha': 1.693305529699566e-05, 'reg_lambda': 3.897250984223176e-06, 'gamma': 0.16970161657112237}
"""

{'num_boost_round': 900, 'learning_rate': 0.011387425172293873, 'max_depth': 9, 'subsample': 0.5642895080936234, 'colsample_bytree': 0.5581569764539639, 'min_child_weight': 19.703853846581026, 'reg_alpha': 3.336634822084743e-07, 'reg_lambda': 1.129627556934382e-07, 'gamma': 0.4896075810838035}
{'num_boost_round': 2300, 'learning_rate': 0.01354184424412271, 'max_depth': 9, 'subsample': 0.7339248014853161, 'colsample_bytree': 0.7407604054011988, 'min_child_weight': 1.392834443288169, 'reg_alpha': 1.693305529699566e-05, 'reg_lambda': 3.897250984223176e-06, 'gamma': 0.16970161657112237}


In [20]:
def train_oof_xgb(y_col, best_params, use_gpu=False):
    y = Y[y_col].values
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(test_X))
    model = []

    num_boost_round = int(best_params.get("n_estimators", 1200))
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "eta": float(best_params["learning_rate"]),
        "max_depth": int(best_params["max_depth"]),
        "subsample": float(best_params["subsample"]),
        "colsample_bytree": float(best_params["colsample_bytree"]),
        "min_child_weight": float(best_params["min_child_weight"]),
        "alpha": float(best_params["reg_alpha"]),
        "lambda": float(best_params["reg_lambda"]),
        "gamma": float(best_params["gamma"]),
        "tree_method": "gpu_hist" if use_gpu else "hist",
        "seed": RANDOM_STATE,
    }

    dtest = xgb.DMatrix(test_X)

    for fold, (tr_idx, va_idx) in enumerate(get_cv().split(X, y)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        dtr = xgb.DMatrix(X_tr, label=y_tr)
        dva = xgb.DMatrix(X_va, label=y_va)

        bst = xgb.train(
            params=params,
            dtrain=dtr,
            num_boost_round=num_boost_round,
            evals=[(dva, "valid")],
            early_stopping_rounds=EARLY_STOP,
            verbose_eval=False,
        )

        oof[va_idx] = _predict_best(bst, dva)
        test_pred += _predict_best(bst, dtest) / N_SPLITS
        model.append(bst)
        print(f"[{y_col}] Fold {fold+1}/{N_SPLITS} complete.")

    cv_r2 = r2_score(Y[y_col], oof)
    rmse = float(np.sqrt(mean_squared_error(Y[y_col], oof)))
    print(f"[OOF] {y_col}: R2={cv_r2:.5f} | RMSE={rmse:.5f}")
    return model, oof, test_pred, cv_r2

In [21]:
xgb_models = {}
oof_xgb = {}
test_xgb = {}
xgb_cv_scores = {}

for tgt in TARGETS:
    best_params_xgb = xgb_studies[tgt].best_trial.params
    models, oof, te, r2cv = train_oof_xgb(tgt, best_params_xgb, use_gpu=False)
    xgb_models[tgt] = models
    oof_xgb[tgt] = oof
    test_xgb[tgt] = te
    xgb_cv_scores[tgt] = r2cv

print("XGB CV R2:", xgb_cv_scores)

[Y1] Fold 1/5 complete.
[Y1] Fold 2/5 complete.
[Y1] Fold 3/5 complete.
[Y1] Fold 4/5 complete.
[Y1] Fold 5/5 complete.
[OOF] Y1: R2=0.76555 | RMSE=0.46999
[Y2] Fold 1/5 complete.
[Y2] Fold 2/5 complete.
[Y2] Fold 3/5 complete.
[Y2] Fold 4/5 complete.
[Y2] Fold 5/5 complete.
[OOF] Y2: R2=0.79521 | RMSE=0.41800
XGB CV R2: {'Y1': 0.7655471242044433, 'Y2': 0.795214747046997}


In [33]:
oof_cat = oof_preds
test_cat = test_preds

def rmse_score(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# simple 50 50 blend
oof_blend_avg = {}
test_blend_avg = {}
for tgt in TARGETS:
    oof_blend_avg[tgt] = 0.5 * oof_cat[tgt] + 0.5 * oof_xgb[tgt]
    test_blend_avg[tgt] = 0.5 * test_cat[tgt] + 0.5 * test_xgb[tgt]
    r2 = r2_score(Y[tgt], oof_blend_avg[tgt])
    rmse = rmse_score(Y[tgt], oof_blend_avg[tgt])
    print(f"[BLEND AVG] {tgt}: R2={r2:.5f} | RMSE={rmse:.5f}")

# ridge-learned wieghts 
weights = {}
oof_blend_ridge = {}
test_blend_ridge = {}

for tgt in TARGETS:
    Xb_oof = np.vstack([oof_cat[tgt], oof_xgb[tgt]]).T
    ridge = Ridge(alpha=1e-3, fit_intercept=True, random_state=RANDOM_STATE)
    ridge.fit(Xb_oof, Y[tgt])
    w_cat, w_xgb = ridge.coef_
    b = ridge.intercept_
    weights[tgt] = {"w_cat": float(w_cat), "w_xgb": float(w_xgb), "intercept": float(b)}

    oof_blend_ridge[tgt] = Xb_oof @ np.array([w_cat, w_xgb]) + b
    print(f"[Ridge Blend] {tgt}: OOF R2={r2_score(Y[tgt], oof_blend_ridge[tgt]):.5f} | RMSE={rmse_score(Y[tgt], oof_blend_ridge[tgt]):.5f} | Weights={weights[tgt]}")

    # Apply to test
    Xb_test = np.vstack([test_cat[tgt], test_xgb[tgt]]).T
    test_blend_ridge[tgt] = Xb_test @ np.array([w_cat, w_xgb]) + b


[BLEND AVG] Y1: R2=0.76480 | RMSE=0.47073
[BLEND AVG] Y2: R2=0.79692 | RMSE=0.41626
[Ridge Blend] Y1: OOF R2=0.76599 | RMSE=0.46954 | Weights={'w_cat': 0.16396229140701432, 'w_xgb': 0.8518348675420883, 'intercept': 0.0003214344310655294}
[Ridge Blend] Y2: OOF R2=0.79709 | RMSE=0.41609 | Weights={'w_cat': 0.3961712310883866, 'w_xgb': 0.6101664135273175, 'intercept': -0.0026380789592869713}


In [37]:
rows = []
for tgt in TARGETS:
    rows.append({
        "target": tgt,
        "CatBoost OOF R2": r2_score(Y[tgt], oof_cat[tgt]),
        "XGBoost OOF R2": r2_score(Y[tgt], oof_xgb[tgt]),
        "Avg(0.5/0.5) OOF R2": r2_score(Y[tgt], oof_blend_avg[tgt]),
        "Ridge Blend OOF R2": r2_score(Y[tgt], oof_blend_ridge[tgt]),
    })
pd.DataFrame(rows).sort_values("Ridge Blend OOF R2", ascending=True)

,target,CatBoost OOF R2,XGBoost OOF R2,Avg(0.5/0.5) OOF R2,Ridge Blend OOF R2
0,Y1,0.759486,0.765547,0.764805,0.765993
1,Y2,0.792794,0.795215,0.796920,0.797087


In [38]:
submission = pd.DataFrame()

submission[ID_COL] = df_test[ID_COL]
for tgt in TARGETS:
    submission[tgt] = test_blend_ridge[tgt]

OUT_PATH = "submission_catboost_xgboost_blend.csv"
submission.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)
submission.head()

Saved: submission_catboost_xgboost_blend.csv


,id,Y1,Y2
0,1,0.512310,-0.159681
1,2,-0.205990,-0.344814
2,3,-0.304852,-0.101092
3,4,-0.443897,0.115019
4,5,-0.944820,0.113564
